In [49]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [50]:
!ls

clean_meas.csv
clean_measurements.csv
clean_station.csv
clean_stations.csv
climate_analysis.ipynb
data_engineering.ipynb
database_engineering.ipynb
hawaii.sqlite
images
merged_hawaii.csv
README.md
Resources
sqlalchemy


In [51]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [52]:
Base = automap_base()

In [53]:
Base.prepare(engine, reflect = True)

In [54]:
Base.classes.keys()

['measurement', 'station']

In [55]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [56]:
session = Session(engine)

In [57]:
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1d23477d7f0>,
 'elevation': 3.0,
 'id': 0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [58]:
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1d234783278>,
 'date': '2010-01-01',
 'id': 0,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65.0}

In [59]:
import pandas as pd

In [65]:
dates = session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).limit(1050).all()
pd.DataFrame(dates)

,date,prcp
0,2017-08-23,0.08
1,2017-08-23,0.45
2,2017-08-22,0.50
3,2017-08-21,0.02
4,2017-08-21,0.56
5,2017-08-20,0.01
6,2017-08-19,0.09
7,2017-08-18,0.06
8,2017-08-17,0.05
9,2017-08-17,0.01


In [61]:
session.query(ObjectRes).order_by(ObjectRes.id.desc()).first()

NameError: name 'ObjectRes' is not defined

In [ ]:
'''
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_temps('2012-02-28', '2012-03-05'))

'''